In [1]:
import os 
import glob
import PIL.Image
from typing import List, Optional, Union
import numpy as np
import torch
from metrics import Metrics, preprocess_image


def load_images_from_dir(image_dir: str):
    all_images = glob.glob(os.path.join(image_dir, "*.png"))
    all_images = [preprocess_image(PIL.Image.open(image)) for image in all_images]
    torch_image_tensor = torch.tensor(np.array(all_images), dtype=torch.float32)
    # (num_frames, channels, height, width)
    torch_image_tensor = torch_image_tensor.permute(0, 3, 1, 2)
    target = torch_image_tensor.to(device)
    return target

/mnt/damian/miniconda3/envs/qa_lib/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device="cuda"
test_image_dir = "data/2c21b97ff3dc4fc3b1ef9e4bb0164318"

metrics = Metrics(device=device)
target = load_images_from_dir(test_image_dir)

/mnt/damian/miniconda3/envs/qa_lib/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/mnt/damian/miniconda3/envs/qa_lib/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `InceptionScore` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)  # noqa: B028
/mnt/damian/miniconda3/envs/qa_lib/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `Kernel Inception Distance` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)  # noqa: B028


In [3]:
for i in range(10):
    input = target + torch.randn_like(target) * 0.001
    input = torch.clamp(input, 0, 1).to(device)
    metrics.compute_image(input, target)
metrics.get_total_metrics()

{'MSE': tensor(233671.2344, device='cuda:0'),
 'CLIP-S': tensor(0.6683, device='cuda:0'),
 'Spectral_MSE': tensor(7.7526e+10, device='cuda:0'),
 'D_lambda': tensor(47048708., device='cuda:0'),
 'ERGAS': tensor(25.6390, device='cuda:0'),
 'PSNR': tensor(0.4657, device='cuda:0'),
 'RASE': tensor(6518.5981, device='cuda:0'),
 'RMSE_wind': tensor(471.0640, device='cuda:0'),
 'SAM': tensor(0.0358, device='cuda:0'),
 'MS-SSIM': tensor(0.3656, device='cuda:0'),
 'SSIM': tensor(0.0094, device='cuda:0'),
 'UQI': tensor(0.0018, device='cuda:0'),
 'VIF': tensor(0.0002, device='cuda:0'),
 'LPIPS': tensor(0.6449, device='cuda:0'),
 'SCC': tensor(0.0028, device='cuda:0'),
 'FID': tensor(8.8580, device='cuda:0'),
 'IS': tensor(0.1628, device='cuda:0'),
 'KID': tensor(0.0418, device='cuda:0')}